In [ ]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import pandas as pd
import numpy as np

In [ ]:
# load data
# import data
df = pd.read_csv('TA_data.csv')

# separate the inputs and target
X = df.drop('timestamp', axis=1)

# create binary classification for price movement
price = pd.DataFrame()
price['today'] = df['price-ohlc-usd-c']
price['next day'] = price['today'].shift(-1)
y = (price['next day'] > price['today']).astype(int)

# convert DataFrame to numpy array
X_np = X.values.copy()
y_np = y.values.copy()

In [ ]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
feat_selector.fit(X_np, y_np)

# check selected features
features_bool = feat_selector.support_
feature_ranking = feat_selector.ranking_
features = np.array(X.columns)
features_selected = features[features_bool]
features_selected_tentative = features[feature_ranking <= feat_selector.max_iter]

In [ ]:
print('Features selected:', features_selected)
features_selected = np.insert(features_selected, 0, 'timestamp')
df[features_selected].to_csv('Boruta_TA_data.csv', index=False)

In [ ]:
df[features_selected]